In [11]:
import numpy as np
import pyldpc
import matplotlib.pyplot as plt
from fractions import Fraction
from time import time
from scipy.sparse import csr_matrix

# I - Construction 
## I - 1 802.11 standard 

In [2]:
Z = 42
_M = {}
_M[Fraction(1, 2)] = [
    [40, 0, 38, 0, 13, 0, 5, 0, 18, 0, 0, 0, 0, 0, 0, 0],
    [34, 0, 35, 0, 27, 0, 0, 30, 2, 1, 0, 0, 0, 0, 0, 0],
    [0, 36, 0, 31, 0, 7, 0, 34, 0, 10, 41, 0, 0, 0, 0, 0],
    [0, 27, 0, 18, 0, 12, 20, 0, 0, 0, 15, 6, 0, 0, 0, 0],
    [35, 0, 41, 0, 40, 0, 39, 0, 28, 0, 0, 3, 28, 0, 0, 0],
    [29, 0, Z, 0, 0, 22, 0, 4, 0, 28, 0, 27, 0, 23, 0, 0],
    [0, 31, 0, 23, 0, 21, 0, 20, 0, 0, 12, 0, 0, Z, 13, 0],
    [0, 22, 0, 34, 31, 0, 14, 0, 4, 0, 0, 0, 13, 0, 22, 24],
]

_M[Fraction(5, 8)] = [
    [20, 36, 34, 31, 20, 7, 41, 34, 0, 10, 41, 0, 0, 0, 0, 0],
    [30, 27, 0, 18, 0, 12, 20, 14, 2, 25, 15, 6, 0, 0, 0, 0],
    [35, 0, 41, 0, 40, 0, 39, 0, 28, 0, 0, 3, 28, 0, 0, 0],
    [29, 0, Z, 0, 0, 22, 0, 4, 0, 28, 0, 27, 24, 23, 0, 0],
    [0, 31, 0, 23, 0, 21, 0, 20, 0, 9, 12, 0, 0, Z, 13, 0],
    [0, 22, 0, 34, 31, 0, 14, 0, 4, 0, 0, 0, 0, 0, 22, 24],
]

_M[Fraction(3, 4)] = [
    [35, 19, 41, 22, 40, 41, 39, 6, 28, 18, 17, 3, 28, 0, 0, 0],
    [29, 30, Z, 8, 33, 22, 17, 4, 27, 28, 20, 27, 24, 23, 0, 0],
    [37, 31, 18, 23, 11, 21, 6, 20, 32, 9, 12, 29, 0, Z, 13, 0],
    [25, 22, 4, 34, 31, 3, 14, 15, 4, 0, 14, 18, 13, 13, 22, 24],
]

_M[Fraction(13, 16)] = [
    [29, 30, Z, 8, 33, 22, 17, 4, 27, 28, 20, 27, 24, 23, 0, 0],
    [37, 31, 18, 23, 11, 21, 6, 20, 32, 9, 12, 29, 10, Z, 13, 0],
    [25, 22, 4, 34, 31, 3, 14, 15, 4, 2, 14, 18, 13, 13, 22, 24],
]

E = np.eye(Z)


def _convert_row(row, e):
    erow = [np.roll(e, i, axis=1) for i in row]
    erow = np.hstack(erow)
    return erow


def _convert_mat(_M, e):
    mat = np.vstack([_convert_row(row, e) for row in _M])
    return mat


H_matrices = {}
for k, v in _M.items():
    H_matrices[k] = _convert_mat(v, E)


### Put the matrices in a list of tuples (H,tG):

In [135]:
H_std = list(H_matrices.values())
std = []
for H in H_std:
    std.append(pyldpc.CodingMatrix_systematic(H))


## I - 2 Callager Regular Codes 
In order to generate regular matrices with the same properties, let's print the shape and number of ones per row/column of the matrices above: 

In [4]:
for H,tG in std:
    print("n,k: {} \n d_v: {} \n d_c: {}\n".format(tG.shape,sum(H)[0],sum(H.T)[0]))

n,k: (672, 343) 
 d_v: 8.0 
 d_c: 16.0

n,k: (672, 425) 
 d_v: 6.0 
 d_c: 16.0

n,k: (672, 507) 
 d_v: 4.0 
 d_c: 16.0

n,k: (672, 548) 
 d_v: 3.0 
 d_c: 16.0



In [15]:
reg = []

n = 672
d_v = 8
d_c = 16
H = pyldpc.RegularH(n,d_v,d_c)
H,tG = pyldpc.CodingMatrix_systematic(H)
reg.append((csr_matrix(H),tG))
tG.shape 

(672, 343)

In [16]:
n = 672
d_v = 6
d_c = 16
H = pyldpc.RegularH(n,d_v,d_c)
H,tG = pyldpc.CodingMatrix_systematic(H)
reg.append((csr_matrix(H),tG))

tG.shape

(672, 425)

In [17]:
n = 672
d_v = 4
d_c = 16
H = pyldpc.RegularH(n,d_v,d_c)
H,tG = pyldpc.CodingMatrix_systematic(H)
reg.append((csr_matrix(H),tG))
tG.shape

(672, 507)

In [18]:
n = 672
d_v = 3
d_c = 16
H = pyldpc.RegularH(n,d_v,d_c)
H,tG = pyldpc.CodingMatrix_systematic(H)
reg.append((csr_matrix(H),tG))
tG.shape

(672, 548)

### Before any computations, let's make sure our lists of matrices match (in shape): 

In [114]:
for (H,tG) in reg:
    print(tG.shape)
for (H,tG) in std:
    print(tG.shape)

(672, 343)
(672, 425)
(672, 507)
(672, 548)
(672, 343)
(672, 425)
(672, 507)
(672, 548)


## Simulation, BER calculations

In [197]:
SNR = [0.5*i for i in range(-10,13)]
max_iter = 50
sample = 50
BER_std = [] #List of BER lists- standard codes
BER_reg = [] # regular codes 

In [207]:
REGULAR = 1
STANDARD = 1

t = time()
for i,(Hstd,tGstd) in enumerate(std):
    Hreg,tGreg = reg[i] ###the regular matrices correspondant tuple 

    n,k = tGstd.shape
    BERi_std=[] #BER-list of one curve
    BERi_reg=[] 

    sample = np.random.randint(2,size=(N,k))
    for snr in SNR:
        BERj_std=[] #BER-statiscal numbers of one dot
        BERj_reg=[] 

        ### STANDARD:
        for j in range(N):
            coded_sample = pyldpc.Coding(tGstd,sample[j],snr)
            decoded_sample = pyldpc.decodingfunctions.Decoding_logBP(Hstd,coded_sample,snr,max_iter)
            decoded_sample = decoded_sample[:k]
            BERj_std.append(sum(abs(decoded_sample-sample[j]))/k)

            coded_sample = pyldpc.Coding(tGreg,sample[j],snr)
            decoded_sample = pyldpc.decodingfunctions.Decoding_logBP(Hreg,coded_sample,snr,max_iter)
            decoded_sample = decoded_sample[:k]
            BERj_reg.append(sum(abs(decoded_sample-sample[j]))/k)

        BERi_std.append(sum(BERj_std)/N)
        BERi_reg.append(sum(BERj_reg)/N)

    BER_std.append(BERi_std)
    BER_reg.append(BERi_reg)

    print("------- \n number {}. \n time = {}".format(i,time()-t))


### Saving lists - joining curve data
The calculations above took many hours, I had to compute them step by step ...

In [196]:
np.savetxt("../Graphs/BER_std_final.txt",BER_std_)
np.savetxt("../Graphs/BER_reg_final.txt",BER_reg_)

In [140]:
BER_std_h = np.loadtxt("../Graphs/BER_std_6.txt")
BER_reg_h = np.loadtxt("../Graphs/BER_reg_6.txt")
BER_std_l = np.loadtxt("../Graphs/BER_std_lowsnr.txt")
BER_reg_l = np.loadtxt("../Graphs/BER_reg_lowsnr.txt")
BER_std_m = np.loadtxt("../Graphs/BER_std_highsnr.txt")
BER_reg_m = np.loadtxt("../Graphs/BER_reg_highsnr.txt")
BER_std_m = np.loadtxt("../Graphs/BER_std_highsnr.txt")
BER_std_pos = np.loadtxt("../Graphs/BER_std_pos.txt")


In [141]:
BER_std_=[]
for i,low in enumerate(BER_std_l):
    BER_std_.append(np.concatenate((low,BER_std_pos[i],)))

BER_reg_=[]
for i,low in enumerate(BER_reg_l):
    BER_reg_.append(np.concatenate((low,BER_reg_m[i],BER_reg_h[i])))
SNR = [0.5*i for i in range(-10,13)]


## PLOTS 

In [199]:

plt.title('Bit Error Rate vs Signal Noise Ratio - iter = '+str(max_iter)+'- sample='+str(N))
plt.plot(SNR, BER_reg_[0], 'b--', label='1/2- regular')
plt.plot(SNR, BER_reg_[1], 'k--', label='5/8- regular')
plt.plot(SNR, BER_reg_[2], 'g--', label='3/4- regular')
plt.plot(SNR, BER_reg_[3], 'r--', label='13/16- regular')

plt.plot(SNR, BER_std_[0], 'b', label='1/2- 802.11')
plt.plot(SNR, BER_std_[1], 'k', label='5/8- 802.11')
plt.plot(SNR, BER_std_[2], 'g', label='3/4- 802.11')
plt.plot(SNR, BER_std_[3], 'r', label='13/16- 802.11')

plt.legend(loc='upper right')
plt.xlim([-5,10])


plt.yscale('log')
plt.grid()
plt.xlabel('SNR in db')
plt.ylabel('BER')
plt.show()


In [205]:

plt.title('Bit Error Rate - Signal Noise Ratio - iter = '+str(max_iter)+'- sample='+str(N))


plt.plot(SNR, BER_std_[0], 'b', label='1/2- 802.11')
plt.plot(SNR, BER_std_[1], 'k', label='5/8- 802.11')
plt.plot(SNR, BER_std_[2], 'g', label='3/4- 802.11')
plt.plot(SNR, BER_std_[3], 'r', label='13/16- 802.11')

plt.legend(loc='upper right')
plt.xlim([-5,0])
plt.ylim([0.15,0.3])

#plt.yscale('log')
plt.grid()
plt.xlabel('SNR in db')
plt.ylabel('BER')
plt.show()


In [203]:

plt.title('Bit Error Rate vs Signal Noise Ratio - iter = '+str(max_iter)+'- sample='+str(N))
plt.plot(SNR, BER_reg_[0], 'b--', label='1/2- regular')
plt.plot(SNR, BER_reg_[1], 'k--', label='5/8- regular')
plt.plot(SNR, BER_reg_[2], 'g--', label='3/4- regular')
plt.plot(SNR, BER_reg_[3], 'r--', label='13/16- regular')



plt.legend(loc='upper right')
plt.xlim([-5,10])

plt.yscale('log')
plt.grid()
plt.xlabel('SNR in db')
plt.ylabel('BER')
plt.show()
